# Radio Communication Via a Computer Interface Test


Now that you have gotten your radio and radio interfaces, we are ready to experiment with them. In this notebook we will test the interface and the radio, make sure that everything is working correctly so that you will be able to make progress on the rest of labs as well as the project.

<center><img src="./radio_interface.jpg" alt="gsm" style="width: 400px;"/></center>
<center>Figure 1: The radio computer interface </center>

The interface you got connects the radio to the sound extension of your Pi through two audio cables. You will send and receive audio from your radio, and you will be able to key the radio from your python script using one of the General Purpose IO (GPIO) pins. The interface also has a ground-loop isolation box. It has two audio transformers that prevent voltage potential from your Pi to get to the radio. The isolation box has a Kenwood style audio connector with a 2.5mm and 3.5mm audio Jack that connects to your Baofeng radio. 

<center><img src="./radio_attenuator.jpg" alt="gsm" style="width: 400px;"/></center>
<center>Figure 2: Connect RF attenuator between the radio and its antenna to prevent RF interference</center>
Since the radio is physically close to our Pi/sound extension and audio interface, the transmitted RF wave could easily cause RF interference with the circuits. We need to install a RF attenuator at its transmission port (between the radio and its antenna shown in Fig.2) to prevent interference. Please keep the attenuator on the radio throughout this lab.



**Starting:**

* Make sure that the channel you will use is not already in use by someone else
* Choose one of the experimental channels (71-98) in the programmed channels on your radio, preferably a UHF one (75-98)
* Before you start transmitting, you must identify with your call sign and that you intend to use this channel for performing experiments
* Connect the audio interface to the radio and to the sound extension of your Pi
* Make sure in alsamixer settings that the volume is properly set to prevent overloading the radio input and that both input and output are not on mute. 
* Make sure the output volume on the radio is reasonable to prevent overloading of the interface. Turning clockwise when the radio is off for 1/4 of a turn works well for me. 

**During operation:**

* If the green light on the radio turns on frequently, it means that someone is transmitting on your channel. In that case, switch to a different one. 
* If the red light turns on, the radio is transmitting. If it is not supposed to, then something got messed up with the sound extension that affects proper radio triggering. It's probably went AWOL due to RF interference. Turn off the radio and reboot the Raspberry Pi with ```sudo reboot```. Turn on the radio and run the code again.
* Announce your callsign every 10 minutes
* Make sure that the antenna is not close to the audio cable and is as orthogonal as possible to it  -- this will prevent RF to get into the audio interface. 
* Sometimes, the sound extension will stop working due to RF overloading. To prevent that, put your radio further away as possible from the interface.
* Use low-power (# key) when possible

**Finishing:**

* Sign off with your call sign

** RF interference from radio transmission that makes the sound extension fail is \#1 cause of technical issues in this lab. If that happens **
* Make sure the antenna is not aligned with any cable
* Make sure the radio is far as possible from the interface
* Put your radio on a shelf or elevated
* Ask us for more Ferrite beeds to put on your cables. 
* For this lab, remove the antenna from the SDR -- since you will be transmitting really close to the SDR, you don't want to overload the receiver.  



## Testing the  Audio:

The first test/example would be to see if we can capture audio from the radio and play it on the raspberry pi.

* Make sure the Pi, Radio, Radio interface is connected as Fig. 1 shows.
* Connect the audio interface to the radio and to the sound extension of your Pi. Connect ```RX AUDIO TO PC``` to ```LINE IN``` and ```TX AUDIO FROM PC``` to ```LINE OUT```
* Connect the speaker to the builtin audio output of the raspberry pi.
* On the radio, press on the orange button to switch to FM radio mode. Tune to 94.1 KPFA. Set the volume at quarter of a turn. You can also tune to NOAA Weather 162.400MHz. 
* The following code records the audio that is coming out of the radio into the sound extension and plays it on the computer built-in speakers. 

In [ ]:
# configure audio settings for the raspberry pi

!amixer -c 1 -- sset 'Capture Mux' 'LINE_IN'
!amixer -c 1 -- sset Lineout playback unmute
!amixer -c 1 -- sset Lineout playback  50%,50%
!amixer -c 1 -- sset Mic capture  67%
!amixer -c 1 -- sset Mic playback  59%


In [ ]:
# Package import
import numpy as np
import matplotlib.pyplot as plt
import queue as Queue
import sounddevice as sd
import RPi.GPIO as GPIO
import time

from testing_functions import myspectrogram_hann_ovlp
%matplotlib inline

In [ ]:
# select built-in audio to play audio / select Fe-Pi audio to record
builtin_idx = 0
USB_idx = 2

# set default sample rate and number of channels. 
fs = 48000
sd.default.samplerate=fs
sd.default.channels = 1

In [ ]:
# this callback function will play captured data 
# it will be called by the soundevice stream and run in a different thread

def replay_callback(indata,outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata  # the [:] is important so data is copied not referenced !
    
 
# create stream
# Will record from device 5 and play through device 3 
st = sd.Stream(device=(USB_idx,builtin_idx),callback=replay_callback)


# start stream -- will run in background till stopped
st.start()

# sleep 15 seconds
time.sleep(15)

# stop and close stream -- must stop and close for clean exit

st.stop()
st.close()

#### If you cannot hear the audio from the radio, adjust the volume knob on the radio to raise the volume until you hear it.

## Testing  Radio Transmit control

* Follow the preparation steps listed in the beginning of the document (identify your callsign and announce testing)
* Pick an experimental channel 71-98
* Set the power of the radio to low if it's not set already (short press the # key. shows up as a small 'L')

* Connect the jumper cables from the interface to the GPIO pins. Connect the ground to Pin number 9, and the other to pin number 16

<center><img src="./pinout.jpg" alt="gsm" style="width: 200px;"/></center>
<center>Figure 3: Raspberry pi pinout</center>

* Connected the second Pi, interface, and radio to listen to the transmission
2) Tune both radio to the same communication channel 
3) Run the following sections

You should be able to hear the transmitting pure-tone signal from the second radio. 


## WARNING: This is where things may fail due to RF interference. If your radio does not stop transmitting, turn off the radio and reboot the Pi. Look above for tips for solving interference problems. Make sure the attenuator is connected to the antenna port of the radio to prevent RF interference **

#### Transmission part of code

In [ ]:
GPIO.cleanup()
GPIO.setmode(GPIO.BOARD)
PTT = 16
GPIO.setup(PTT, GPIO.OUT, initial = 0)

The following cell is for transmission of a pure tone. Pi #1 should run this while Pi #2 should run the receiving part of the code at the same time to listen to the transmitted signal

In [ ]:
t = np.r_[0:1*48000]/48000
sig1 = 0.5*np.sin(2*np.pi*2000*t)
sig2 = 0.5*np.sin(2*np.pi*1000*t)

GPIO.output(PTT, GPIO.HIGH) # Key radio
time.sleep(0.1) #give radio time to start
sd.play(sig1,device=USB_idx,  blocking=True)  
GPIO.output(PTT, GPIO.LOW)

time.sleep(0.1)

GPIO.output(PTT, GPIO.HIGH)
time.sleep(0.1) #give radio time to start
sd.play(sig2,samplerate=48000,device=USB_idx,  blocking=True)
GPIO.output(PTT, GPIO.LOW)

If the radio is stuck and continues to transmit signal, this means the Pi is affected by RF interference. <br/>
1) Turn off the radio and stop the program. <br/>
2) Run the following code to set the GPIO to low. <br/> 
3) After this, reboot the Pi to reset the GPIO setting. <br/>

In [ ]:
GPIO.output(PTT, GPIO.LOW)

#### Receiving part of code
Run this part of the code with Pi #2 to listen to the FM signal transmitted from Pi #1.

In [ ]:
def queuereplay_callback(indata,outdata, frames, time, status):
    if status:
        print(status)
    outdata[:] = indata
    Qin.put( indata )  # Global queue

Qin = Queue.Queue()

# Will record from device 2 and play through device 0
st = sd.Stream( device=(USB_idx, builtin_idx),callback=queuereplay_callback)

st.start()

# record and play about 10.6 seconds of audio 1000*512/48000 = 10.6 s

T_record = 3 # record 3 sec

Nseg = T_record * fs // 512
sig_record = np.zeros(Nseg*512)

for n in range(0,Nseg):
    
    samples = Qin.get()
    sig_record[512*n:512*(n+1)]=samples.reshape((512,))
    
st.stop()
st.close()

# empty queue just in case there's something left
while not(Qin.empty()) :
    samples=Qin.get()

Plot the result of received signal. The received signal is supposed to look like the plots below:
<center><img src="./Normal_sequence.png" alt="gsm" style="width: 400px;"/></center>
<center>Figure 4: Normal recorded sequence</center>
<center><img src="./Normal_spectrogram.png" alt="gsm" style="width: 400px;"/></center>
<center>Figure 5: Normal spectrogram of recorded sequence</center>

Note that if the volume of the listening radio is too large, you will saturate the recording device. Then, you will get something like below. Try to tune down the radio volume to avoid this happening.
<center><img src="./Saturated_sequence.png" alt="gsm" style="width: 400px;"/></center>
<center>Figure 6: Saturated recorded sequence</center>
<center><img src="./Saturated_spectrogram.png" alt="gsm" style="width: 400px;"/></center>
<center>Figure 7: Saturated spectrogram of recorded sequence</center>



In [ ]:
# Plotting the recorded sequence

fig = plt.figure(figsize=(16,4))
t = np.r_[0:Nseg*512]/48000
plt.plot(t,sig_record)
plt.title('Recorded sequence')
plt.xlabel('Time (sec)')
plt.ylabel('Amplitude')



In [ ]:
# Plotting the spectrogram of the received sequence
tt,ff,xmf = myspectrogram_hann_ovlp(sig_record, 512, fs, 0 ,dbf = 40)
plt.title('Spectrogram of the recorded sequence')